In [ ]:
# StewartStack
# Data Cleaning and ML learning for CS 
# 12/04/2023

### === Understanding the Data ===


### === The Dataset ===

#### === Importing libraries & creating settings ===

In [135]:
# Import the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Enable the display of matplotlib plots directly in the Jupyter Notebook
%matplotlib inline

# Set the default figure size for matplotlib plots
plt.rcParams['figure.figsize'] = (12, 8)

# Set the maximum number of columns to display in a DataFrame to 120
pd.set_option('display.max_columns', 120)

# Set the maximum width of each column to display in a DataFrame to 5000 characters
pd.set_option('display.max_colwidth', 5000)


In [ ]:
# Loading in csv file and using skiprow()
# Row 1 contains a link & sits above the series headings.
# Removing so the data can be parsed properly
loans_2007 = pd.read_csv('lending_club_loans.csv', skiprows = 1, low_memory = False)

In [ ]:
loans_2007.head(3)

#### === Thresh ===

In [ ]:
# Remove all columns with more than 50% missing values & 2 columns which are not needed
# Reassign in both cases
# Calculate the half of the total number of rows in the 'loans_2007' DataFrame
half_count  = len(loans_2007) / 2
# Drop the columns in the 'loans_2007' DataFrame that have more than half_count missing values
loans_2007 = loans_2007.dropna(thresh = half_count, axis = 1)
# Drop the 'url' and 'desc' columns from the 'loans_2007' DataFrame
loans_2007 = loans_2007.drop(['url', 'desc'], axis = 1)

In [ ]:
# checking the shape
loans_2007.shape

In [ ]:
# checking the columns count again
loans_2007.head(3)

In [ ]:
# checking info
loans_2007.info()

### === Data Dictionary ===

In [ ]:
# read the DD, check number of rows, get column names
data_dictionary = pd.read_csv('LCDataDictionary.csv')
print(data_dictionary.shape[0])
print(data_dictionary.columns.tolist())

In [ ]:
# Renaming series for clarity and consistency
data_dictionary = data_dictionary.rename(columns={'LoanStatNew': 'name', 'Description': 'description'})

In [ ]:
# Viewing the results
data_dictionary.head()

In [ ]:
# merge()
# merge the data dictionary with part of loans. Assign it new variable
# Create a DataFrame of the data types of columns in the 'loans_2007' DataFrame
loans_2007_dtypes = pd.DataFrame(loans_2007.dtypes, columns=['dtypes'])

# Reset the index of the 'loans_2007_dtypes' DataFrame and add the index as a new column named 'name'
loans_2007_dtypes = loans_2007_dtypes.reset_index()
loans_2007_dtypes['name'] = loans_2007_dtypes['index']

# Rearrange the columns in the 'loans_2007_dtypes' DataFrame
loans_2007_dtypes  = loans_2007_dtypes[['name', 'dtypes']]

# Add a new column 'first value' to the 'loans_2007_dtypes' DataFrame, containing the first value of each column in the 'loans_2007' DataFrame
loans_2007_dtypes['first value'] = loans_2007.loc[0].values

# Merge the 'loans_2007_dtypes' DataFrame with the 'data_dictionary' DataFrame, matching on the 'name' column
preview = loans_2007_dtypes.merge(data_dictionary, on='name', how='left')


### === Select Data for Cleaning using Dictionary ===


In [ ]:
# First group of columns
preview[:19]

In [ ]:
loans_2007.shape

In [ ]:
# first drop - 8 columns
# These are not needed
drop_one = ['id','member_id','funded_amnt','funded_amnt_inv','int_rate','sub_grade','emp_title','issue_d']
# Prevents error 
loans_2007 = loans_2007.drop(drop_one, errors='ignore',axis=1)

In [ ]:
loans_2007.shape

In [ ]:
# Second group of columns
preview[19:38]

In [ ]:
# Second drop - 5 columns
drop_two = [ 'zip_code','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv']

loans_2007 = loans_2007.drop(drop_two,errors='ignore',axis=1)

In [ ]:
loans_2007.shape

In [ ]:
# Third group of columns
preview[38:]

In [ ]:
# Third Group of columns to be dropped - 7
drop_three = ['total_rec_prncp','total_rec_int', 'total_rec_late_fee','recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt']

loans_2007 = loans_2007.drop(drop_three, errors = 'ignore', axis = 1)

In [ ]:
loans_2007.shape

In [ ]:
preview[:]

In [ ]:
# Changing title to loan_title as a bit more appropriate
loans_2007.rename(columns = {'title' : 'loan_title'}, inplace = True)
loans_2007.head(3)

### === Investigating FICO Score Columns ===


In [ ]:
# unique FICO's
print('low', loans_2007['fico_range_low'].unique())
print('\nhigh', loans_2007['fico_range_high'].unique())

In [ ]:
# Comparing fico_low max & min
print('fico low max', loans_2007['fico_range_low'].max())
print('fico low min', loans_2007['fico_range_low'].min())

In [ ]:
# comparing fico_high max & min
print('fico high max', loans_2007['fico_range_high'].max())
print('fico high min', loans_2007['fico_range_high'].min())

In [ ]:
# isnull()
# Checking for null values as want the average to be accurate
loans_2007['fico_range_high'].isnull().values.sum(0)

In [ ]:
loans_2007['fico_range_low'].isnull().values.sum(0)

In [ ]:
# Removing the nulls
# creating a variable & assigning the 2 series to it
# Checking the initial number of rows
# drop rows with missing values for fico high/fico low
# check number of rows again
fico_columns = ['fico_range_high', 'fico_range_low']
print('check rows', loans_2007.shape[0])
loans_2007.dropna(subset = fico_columns, inplace = True)
print('check rows after drop', loans_2007.shape[0])

In [ ]:
# calculating the mean of fico high/fico low
loans_2007['fico_average'] = (loans_2007['fico_range_high'] + loans_2007['fico_range_low']) / 2
cols = ['fico_range_low', 'fico_range_high', 'fico_average']
loans_2007[cols].head() 

In [ ]:
# Dropping unneeded fico columns
drop_cols = ['fico_range_low', 'fico_range_high', 'last_fico_range_low', 'last_fico_range_high']
loans_2007 = loans_2007.drop(drop_cols, errors = 'ignore', axis = 1)
loans_2007.shape

#### === Target Column ===

In [ ]:
# Using preview to check the dataframe on the 'loan_status' series
preview[preview.name == 'loan_status']

In [ ]:
# value counts()
# Returning the frequency of the unique values in the loan_status
loans_2007['loan_status'].value_counts()

In [ ]:
# Using replace() and bitwise
# Having to use this because in pandas using OR will produce ValueError
loans_2007 = loans_2007[(loans_2007['loan_status'] == 'Fully Paid') | (loans_2007['loan_status'] == 'Charged Off')]
new_status = {'loan_status' : {'Fully Paid': 1, 'Charged Off': 0}}
loans_2007 = loans_2007.replace(new_status)
loans_2007.shape

In [ ]:
loans_2007.head()

#### === Visualizing the Target Column Outcomes ===

In [ ]:
# Defining how many plots wanted(1 = one row, 2 = 2 columns; set the size for the canvas)
fig, axs = plt.subplots(1, 2, figsize = (14, 7))
# Seaborn bar chart using loan_status for the x-axis, take the data from the dataset, set the position of the canvas
sns.countplot(x = 'loan_status', data = loans_2007, ax = axs[0])
# Naming the area assigned to plot 1
axs[0].set_title('Frequency of each loan Status')
# Creates pie chart using matplotlib (ax = axs[1] places it correctly on the canvas, autopct sets the decimal display)
loans_2007.loan_status.value_counts().plot(x = None, y = None, kind = 'pie', ax = axs[1], autopct = '%1.2f%%')
# Name the area assigned to plot 2
axs[1].set_title('Percentage of each Loan status')
plt.show()


#### === Saving Cleansed Data ===

In [ ]:
# Create a new file called filtered_loans write the changed data to it
loans_2007.to_csv('cleaned_data/filtered_loans_2007.csv', index = False)

In [136]:
# Reading new file back in
filtered_loans = pd.read_csv('cleaned_data/filtered_loans_2007.csv')
print(filtered_loans.shape)
filtered_loans.head()

(39239, 33)


,loan_amnt,term,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,purpose,loan_title,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens,fico_average
0,5000.0,36 months,162.87,B,10+ years,RENT,24000.0,Verified,1,n,credit_card,Computer,AZ,27.65,0.0,Jan-85,1.0,3.0,0.0,13648.0,83.70%,9.0,f,Sep-16,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0,737.0
1,2500.0,60 months,59.83,C,< 1 year,RENT,30000.0,Source Verified,0,n,car,bike,GA,1.00,0.0,Apr-99,5.0,3.0,0.0,1687.0,9.40%,4.0,f,Sep-16,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0,742.0
2,2400.0,36 months,84.33,C,10+ years,RENT,12252.0,Not Verified,1,n,small_business,real estate business,IL,8.72,0.0,Nov-01,2.0,2.0,0.0,2956.0,98.50%,10.0,f,Sep-16,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0,737.0
3,10000.0,36 months,339.31,C,10+ years,RENT,49200.0,Source Verified,1,n,other,personel,CA,20.00,0.0,Feb-96,1.0,10.0,0.0,5598.0,21%,37.0,f,Apr-16,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0,692.0
4,5000.0,36 months,156.46,A,3 years,RENT,36000.0,Source Verified,1,n,wedding,My wedding loan I promise to pay back,AZ,11.20,0.0,Nov-04,3.0,9.0,0.0,7963.0,28.30%,12.0,f,Jan-16,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0,732.0


#### === Preparing for features for Machine Learning ===

#### === Handle Missing Values ===

In [137]:
# Using isnull() to find missing values to then determine how to use them
null_counts = filtered_loans.isnull().sum()
print('Number of null values in each column:\n', null_counts)

Number of null values in each column:
 loan_amnt                        0
term                             0
installment                      0
grade                            0
emp_length                    1057
home_ownership                   0
annual_inc                       0
verification_status              0
loan_status                      0
pymnt_plan                       0
purpose                          0
loan_title                      11
addr_state                       0
dti                              0
delinq_2yrs                      0
earliest_cr_line                 0
inq_last_6mths                   0
open_acc                         0
pub_rec                          0
revol_bal                        0
revol_util                      50
total_acc                        0
initial_list_status              0
last_credit_pull_d               2
collections_12_mths_ex_med      56
policy_code                      0
application_type                 0
acc_now_delinq  

In [138]:
# removing rows with missing values
filtered_loans = filtered_loans.drop('emp_length', axis = 1)
filtered_loans = filtered_loans.dropna()

In [139]:
# Checkpoint 6
filtered_loans.shape

(38478, 32)

#### === Catagorical Columns ===


In [141]:
print('Data types and their frequency\n', (filtered_loans.dtypes.value_counts()))

Data types and their frequency
 float64    18
object     13
int64       1
dtype: int64
